In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch-geometric pandas numpy scikit-learn networkx matplotlib seaborn tqdm -q

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from ucimlrepo import fetch_ucirepo
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.nn import SAGEConv, GATv2Conv, GCNConv
import psutil

try:
    import pynvml
    pynvml.nvmlInit()
    NVML_AVAILABLE = True
except:
    NVML_AVAILABLE = False

/home/klema/miniconda3/envs/graphsage/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Загружаем датасет German Credit.
Содержит 1000 клиентов и 20 признаков (7 числовых и 13 категориальных)
Целевая переменная: "good" (1) или "bad" (2) кредитная история.

In [2]:
german_credit = fetch_ucirepo(id=144)

X = german_credit.data.features
y = german_credit.data.targets 

# Заменяем "good" на 0, "bad" на 1
y = y['class'].values
y = np.where(y == 1, 0, 1)

# Обработка категориальных признаков. pd.get_dummies автоматически кодирует все категориальных колонки. drop_first=True убирает избыточность.
X = pd.get_dummies(X, drop_first=True)

# Стандартизация числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Строим граф методом k-ближайших соседей. Создаётся разреженная матрица смежности с помощью knn. Рёбра только 0/1, петли исключаются. Граф неориентированный.
k = 5
adj_matrix = kneighbors_graph(X_scaled, k, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
edge_index = to_undirected(edge_index)

# train / val / test (60/20/20)
num_nodes = X_scaled.shape[0]
indices = np.arange(num_nodes)

train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

data = Data(
    x=torch.tensor(X_scaled, dtype=torch.float),
    edge_index=edge_index,
    y=torch.tensor(y, dtype=torch.long),
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
)

print(f"Загружено {data.x.shape[0]} узлов, {data.x.shape[1]} признаков")
print(f"Train/Val/Test: {train_mask.sum().item()}/{val_mask.sum().item()}/{test_mask.sum().item()}")
print(f"Рёбер: {edge_index.shape[1]}")

class_counts = torch.bincount(data.y).tolist()
class_dist_str = ", ".join(f"Класс {i}: {count}" for i, count in enumerate(class_counts))
print(f"Распределение классов: {class_dist_str}")

Загружено 1000 узлов, 48 признаков
Train/Val/Test: 600/200/200
Рёбер: 7528
Распределение классов: Класс 0: 700, Класс 1: 300


In [3]:
def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data):
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

class GraphSAGE(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.sage1 = SAGEConv(dim_in, dim_h)
        self.sage2 = SAGEConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.01,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = self.sage1(x, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.sage2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
            val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                              data.y[data.val_mask])

            if(epoch % 10 == 0):
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

class GAT(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=4):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads, dropout=0.6)
        self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=heads, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.005,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(h, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            # Validation
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
            val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                              data.y[data.val_mask])

            if(epoch % 10 == 0):
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

class GCN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                        lr=0.01,
                                        weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.5, training=self.training)
        h = self.gcn1(h, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.gcn2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = self.optimizer

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            _, out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            # Validation
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
            val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                              data.y[data.val_mask])

            if(epoch % 10 == 0):
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def monitor_resources():
    stats = {}
    # CPU & RAM
    stats['ram_mb'] = psutil.virtual_memory().used / (1024 ** 2)
    stats['cpu_percent'] = psutil.cpu_percent()

    # GPU
    if device.type == 'cuda' and NVML_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        stats['gpu_mem_mb'] = mem_info.used / (1024 ** 2)
        stats['gpu_util'] = util.gpu
    else:
        stats['gpu_mem_mb'] = None
        stats['gpu_util'] = None
    return stats

def train_with_monitoring(model, data, epochs, model_name):
    print(f"\n{'='*50}\nTraining {model_name} with resource monitoring\n{'='*50}")
    
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    start_time = time.time()

    model.fit(data, epochs)

    final_ram = psutil.virtual_memory().used / (1024 ** 2)
    max_gpu_mem = None
    if device.type == 'cuda':
        max_gpu_mem = torch.cuda.max_memory_allocated() / (1024 ** 2)  # MB

    duration = time.time() - start_time
    test_acc = test(model, data)

    results = {
        'test_acc': test_acc,
        'training_time_sec': duration,
        'final_ram_mb': final_ram,
        'max_gpu_mem_mb': max_gpu_mem,
    }

    print(f"\n{model_name} finished")
    print(f"Test Accuracy: {test_acc*100:.2f}%")
    print(f"Training Time: {duration:.1f} sec")
    if max_gpu_mem:
        print(f"Peak GPU Memory: {max_gpu_mem:.1f} MB")
    print(f"Final RAM Usage: {final_ram:.1f} MB")

    return test_acc, results

In [5]:
%%time

results = {}

# 1. GraphSAGE
graphsage = GraphSAGE(data.x.shape[1], dim_h=64, dim_out=2).to(device)
acc_sage, _ = train_with_monitoring(graphsage, data, epochs=100, model_name="GraphSAGE")
results['GraphSAGE'] = acc_sage


Training GraphSAGE with resource monitoring
Epoch   0 | Train Loss: 0.687 | Train Acc:  59.00% | Val Loss: 0.70 | Val Acc: 55.00%
Epoch  10 | Train Loss: 0.439 | Train Acc:  80.00% | Val Loss: 0.56 | Val Acc: 71.00%
Epoch  20 | Train Loss: 0.332 | Train Acc:  86.67% | Val Loss: 0.59 | Val Acc: 71.50%
Epoch  30 | Train Loss: 0.287 | Train Acc:  86.83% | Val Loss: 0.68 | Val Acc: 72.50%
Epoch  40 | Train Loss: 0.188 | Train Acc:  92.83% | Val Loss: 0.75 | Val Acc: 74.50%
Epoch  50 | Train Loss: 0.176 | Train Acc:  92.50% | Val Loss: 0.81 | Val Acc: 74.50%
Epoch  60 | Train Loss: 0.167 | Train Acc:  92.83% | Val Loss: 1.01 | Val Acc: 71.50%
Epoch  70 | Train Loss: 0.131 | Train Acc:  94.67% | Val Loss: 1.08 | Val Acc: 69.50%
Epoch  80 | Train Loss: 0.105 | Train Acc:  96.17% | Val Loss: 1.10 | Val Acc: 73.00%
Epoch  90 | Train Loss: 0.093 | Train Acc:  96.83% | Val Loss: 1.33 | Val Acc: 72.00%
Epoch 100 | Train Loss: 0.077 | Train Acc:  97.67% | Val Loss: 1.07 | Val Acc: 73.50%

GraphSAG

In [6]:
%%time

# 2. GCN
gcn = GCN(data.x.shape[1], dim_h=64, dim_out=2).to(device)
acc_gcn, _ = train_with_monitoring(gcn, data, epochs=100, model_name="GCN")
results['GCN'] = acc_gcn


Training GCN with resource monitoring
Epoch   0 | Train Loss: 0.837 | Train Acc:  45.17% | Val Loss: 0.80 | Val Acc: 44.00%
Epoch  10 | Train Loss: 0.557 | Train Acc:  73.83% | Val Loss: 0.62 | Val Acc: 71.00%
Epoch  20 | Train Loss: 0.507 | Train Acc:  74.50% | Val Loss: 0.56 | Val Acc: 72.50%
Epoch  30 | Train Loss: 0.490 | Train Acc:  76.50% | Val Loss: 0.57 | Val Acc: 73.50%
Epoch  40 | Train Loss: 0.480 | Train Acc:  76.00% | Val Loss: 0.55 | Val Acc: 69.50%
Epoch  50 | Train Loss: 0.482 | Train Acc:  76.00% | Val Loss: 0.57 | Val Acc: 73.50%
Epoch  60 | Train Loss: 0.472 | Train Acc:  77.67% | Val Loss: 0.57 | Val Acc: 72.50%
Epoch  70 | Train Loss: 0.459 | Train Acc:  78.33% | Val Loss: 0.60 | Val Acc: 73.00%
Epoch  80 | Train Loss: 0.469 | Train Acc:  78.50% | Val Loss: 0.56 | Val Acc: 74.00%
Epoch  90 | Train Loss: 0.461 | Train Acc:  77.50% | Val Loss: 0.59 | Val Acc: 68.50%
Epoch 100 | Train Loss: 0.458 | Train Acc:  78.33% | Val Loss: 0.60 | Val Acc: 73.00%

GCN finished
T

In [7]:
%%time

# 3. GAT
gat = GAT(data.x.shape[1], dim_h=32, dim_out=2, heads=4).to(device)
acc_gat, _ = train_with_monitoring(gat, data, epochs=100, model_name="GAT")
results['GAT'] = acc_gat


Training GAT with resource monitoring
Epoch   0 | Train Loss: 4.360 | Train Acc:   5.83% | Val Loss: 4.32 | Val Acc: 6.50%
Epoch  10 | Train Loss: 1.166 | Train Acc:  58.83% | Val Loss: 1.19 | Val Acc: 59.50%
Epoch  20 | Train Loss: 0.758 | Train Acc:  67.67% | Val Loss: 0.81 | Val Acc: 60.50%
Epoch  30 | Train Loss: 0.658 | Train Acc:  69.67% | Val Loss: 0.70 | Val Acc: 64.50%
Epoch  40 | Train Loss: 0.646 | Train Acc:  69.83% | Val Loss: 0.64 | Val Acc: 73.00%
Epoch  50 | Train Loss: 0.631 | Train Acc:  70.17% | Val Loss: 0.66 | Val Acc: 70.00%
Epoch  60 | Train Loss: 0.603 | Train Acc:  69.67% | Val Loss: 0.63 | Val Acc: 69.00%
Epoch  70 | Train Loss: 0.614 | Train Acc:  69.67% | Val Loss: 0.63 | Val Acc: 70.00%
Epoch  80 | Train Loss: 0.603 | Train Acc:  69.50% | Val Loss: 0.65 | Val Acc: 66.50%
Epoch  90 | Train Loss: 0.594 | Train Acc:  68.67% | Val Loss: 0.63 | Val Acc: 70.00%
Epoch 100 | Train Loss: 0.572 | Train Acc:  70.50% | Val Loss: 0.63 | Val Acc: 69.00%

GAT finished
Te